In [1]:
import torch
from torchvision import transforms, utils
import pandas as pd
import numpy as np
import csv

import cxr_dataset as CXR
import eval_model as E
import model as M
import argparse


from tqdm import tqdm

d:\machineLearning\miniconda\envs\pytorch-gpu\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
parser = argparse.ArgumentParser()
parser.add_argument('-name', type=str, default="nih_25")
parser.add_argument('--dataset_path', type=str, default="imgdata/images-NIH-224/")
parser.add_argument('--num_epochs', type=int, default=1, help='')
parser.add_argument('--batch_size', type=int, default=32, help='')
parser.add_argument('--lr', type=float, default=0.01, help='')
parser.add_argument('--weight_decay', type=float, default=1e-4, help='')
#parser.add_argument('--weight_decay', type=float, default=1e-4, help='')
cfg = parser.parse_args(args=[]) #add this line to use argparse in Notebook

print(cfg)

Namespace(batch_size=32, dataset_path='imgdata/images-NIH-224/', lr=0.01, name='nih_25', num_epochs=1, weight_decay=0.0001)


In [7]:
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
# define torchvision transforms
data_transforms = {
        'train': transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.Resize(224),
            # because resize doesn't always give 224 x 224, this ensures 224 x
            # 224
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
        'val': transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ]),
    }


In [8]:
checkpoint_best = torch.load(f'results/{cfg.name}_checkpoint')
model = checkpoint_best['model']

In [9]:
 # get preds and AUCs on test fold
preds, aucs = E.make_pred_multilabel(data_transforms, model, cfg)

100%|██████████| 1403/1403 [03:05<00:00,  7.55it/s]


In [10]:
aucs

,label,auc
0,Atelectasis,0.807973
1,Cardiomegaly,0.894176
2,Consolidation,0.801088
3,Edema,0.888980
4,Effusion,0.877533
5,Emphysema,0.916838
6,Fibrosis,0.817929
7,Hernia,0.887679
8,Infiltration,0.704461
9,Mass,0.828798
